# Classifieur binaire

## Importation des packages

In [0]:
import sys      
import urllib
from urllib.request import urlopen
from urllib.error import HTTPError, URLError
from bs4 import BeautifulSoup, Tag, NavigableString, Comment
import pickle
from collections import defaultdict
from sklearn.feature_extraction.text import TfidfVectorizer # ou CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
#from nltk.corpus import stopwords
from google.colab import drive
import tensorflow as tf
import pandas as pd
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
import numpy as np
import os
from sklearn.feature_extraction.text import TfidfVectorizer
import time
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.metrics import accuracy_score
import os
from collections import defaultdict, Counter
import numpy as np
import pandas as pd
import re
import nltk
import spacy
import random
import sklearn.utils
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize, TweetTokenizer
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.dummy import DummyClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix

Using TensorFlow backend.


In [0]:
#!pip install -U spacy
#!python -m spacy download fr_core_news_sm
nlp = spacy.load('fr_core_news_sm')

## Lecture du fichier comportant les faq scrappées et le corpus et création des jeux de données

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authentification Google
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
 
id = '1LpPR8X-0uD0_LfBEdfaz-2pooafNGUnX'
downloaded = drive.CreateFile({'id': '1LpPR8X-0uD0_LfBEdfaz-2pooafNGUnX'})
downloaded.GetContentFile('faq')

faq = pickle.load(open('faq', 'rb'))

In [0]:
texts = []
labels = []
quest = []
rep = []

In [0]:
for cl,questions in faq.items() : 
  for question in questions.items(): 
    labels.append(str(cl))
    texts.append(str(question[0]))
    quest.append(str(question[0]))
    labels.append(str(cl))
    texts.append(str(question[1]))
    rep.append(str(question[1]))

In [0]:
print(len(labels), len(texts))

266 266


In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authentification Google
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

#Pour les données de validation : 
id = '1mwnba7jZFOVy7YT0mCPYGnz7T_GOWYLU'
downloaded = drive.CreateFile({'id': '1mwnba7jZFOVy7YT0mCPYGnz7T_GOWYLU'})
downloaded.GetContentFile('OpenSubtitles.de-fr.fr')

In [0]:
corpus_conv = open("OpenSubtitles.de-fr.fr", "r").readlines()

Les phrases étant plus petites dans le corpus, on prend plus de corpus que de phrases thématiques.

In [0]:
random.seed(9001)
ind = np.random.permutation(len(corpus_conv))
corpus_random = [corpus_conv[idx] for idx in ind[:400]]

In [0]:
label_corpus = np.repeat(0,len(corpus_random))

In [0]:
label_corpus

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [0]:
labels = list(np.repeat(1,len(texts)))
labels.extend(label_corpus)

In [0]:
len(labels)

666

In [0]:
texts.extend(corpus_random)

In [0]:
len(texts)

666

In [0]:
with open("texts", "wb") as fin :
  pickle.dump(texts, fin)

In [0]:
data = pd.DataFrame([texts, labels], index=["texts","labels"])

In [0]:
data = np.transpose(data)

In [0]:
data

,texts,labels
0,Comment réserver mon voyage ?,1
1,"Pour réserver un voyage tui.fr, plusieurs solu...",1
2,Quels sont les moyens de paiement ?,1
3,Différents moyens de paiement sont possibles s...,1
4,Mes coordonnées bancaires sont-elles réellemen...,1
...,...,...
661,"Pas encore, humain. Bientôt...\n",0
662,Gare si tu oublies que tu es mon chou\n,0
663,"- Comme tu voudras, Hi.\n",0
664,Je l'ai annoncé à mes proches l'année dernière...,0


In [0]:
data = sklearn.utils.shuffle(data)

In [0]:
dtrain , dtest = train_test_split(data, test_size=0.15)

In [0]:
sum(dtrain['labels'])/dtrain.shape[0]

0.4081272084805654

In [0]:
sum(dtest['labels'])/dtest.shape[0]

0.35

In [0]:
dtest.shape[0]

100

In [0]:
dtrain['texts'].values[:20]

array(["Cela ne pose pas de problème. Assurez-vous de disposer de tous les papiers médicaux nécessaires durant votre voyage afin d'attester de votre traitement médical auprès de votre compagnie aérienne durant le vol et de votre hôtel durant votre séjour sur place.",
       'Si je suis insatisfait de mon séjour, quelle procédure dois-je suivre ?',
       'Qui dois-je contacter si je rencontre un problème durant mon séjour ?',
       'Et le mari lui a versé une pension durant cette période...\n',
       "Ce n'est pas une question de vie ou de mort.\n",
       'Ca fait environ 30 secondes donc... Combien de temps ?\n',
       'Quoi ?\n', '- Rien derrière nous ?\n',
       "Comment faire pour les déplacements à l'aéroport si je voyage avec une personne handicapée ?",
       'Il constitue une menace.\n',
       'Comment puis-je percevoir le remboursement des taxes aériennes ?',
       'Puis-je joindre un conseiller en cas de problème ou pour d’éventuels renseignements ?',
       "Je l'ai a

## Création des fonctions de traitement de texte

In [0]:
def lemmatise_text(text):
    """ Met à l'infinitif les verbes et met la racine des mots """
    tw_nlp = nlp(text) # spacy
    tokens = []
    for token in tw_nlp:
        tokens.append(token.lemma_)
    return ' '.join(tokens)


In [0]:
def replace_words_with_pos_tag(text):
    exp = []
    text = nlp(text) # spacy
    for token in text:
        exp.append(token.pos_)
    return ' '.join(exp)

In [0]:
def ner(text):
    tw_nlp = nlp(text) # spacy
    clas = []
    for ent in tw_nlp.ents:
        clas.append(ent.label_)
    return ' '.join(clas)

In [0]:
import nltk
nltk.download('stopwords')
sw = nltk.corpus.stopwords.words('french')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [0]:
sw += ['être', 'avoir', 'etre', 'puis' ]

In [0]:
bin_count = CountVectorizer(binary=True,stop_words = sw)

In [0]:
X_train = dtrain['texts']
y_train = list(dtrain['labels'])
X_test = dtest['texts']
y_test = list(dtest['labels'])

In [0]:
bin_count.fit(X_train)
bin_count

CountVectorizer(analyzer='word', binary=True, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None,
                stop_words=['au', 'aux', 'avec', 'ce', 'ces', 'dans', 'de',
                            'des', 'du', 'elle', 'en', 'et', 'eux', 'il', 'ils',
                            'je', 'la', 'le', 'les', 'leur', 'lui', 'ma',
                            'mais', 'me', 'même', 'mes', 'moi', 'mon', 'ne',
                            'nos', ...],
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

## Création des matrices terme document

### Matrice terme document binaire

In [0]:
X_train_vectorized_bin = bin_count.transform(X_train)
X_train_vectorized_bin

<566x1821 sparse matrix of type '<class 'numpy.int64'>'
	with 4767 stored elements in Compressed Sparse Row format>

In [0]:
X_test_vectorized_bin = bin_count.transform(X_test)

### Matrice terme document numérique discret
nombre d'occurence

In [0]:
vect_count =  CountVectorizer(binary=False,stop_words = sw) # # compléter ici : comme plus haut, mais binary=False

In [0]:
vect_count.fit(X_train)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None,
                stop_words=['au', 'aux', 'avec', 'ce', 'ces', 'dans', 'de',
                            'des', 'du', 'elle', 'en', 'et', 'eux', 'il', 'ils',
                            'je', 'la', 'le', 'les', 'leur', 'lui', 'ma',
                            'mais', 'me', 'même', 'mes', 'moi', 'mon', 'ne',
                            'nos', ...],
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [0]:
vect_count.get_feature_names()[:50]

['00',
 '000',
 '01',
 '015',
 '016',
 '03',
 '0825',
 '10',
 '100',
 '10h',
 '11',
 '121',
 '13e',
 '15',
 '15kg',
 '16',
 '18',
 '19',
 '1res',
 '1ères',
 '20',
 '20h',
 '20kg',
 '21',
 '21h30',
 '24',
 '24h',
 '25',
 '2h',
 '30',
 '31',
 '32',
 '32ème',
 '33174180376',
 '39',
 '3h',
 '40',
 '48',
 '48h',
 '50',
 '5000',
 '5kg',
 '72',
 '74',
 '75',
 '76',
 '825',
 '826',
 '911',
 '92309']

In [0]:
vect_count.get_feature_names()[-50:]

['voulait',
 'vouloir',
 'voyage',
 'voyager',
 'voyages',
 'voyageur',
 'voyagez',
 'vraiment',
 'vu',
 'véhicule',
 'vérifier',
 'vérifierons',
 'vérifiez',
 'vôtre',
 'wang',
 'wes',
 'woods',
 'www',
 'zippy',
 'zorro',
 'zut',
 'ça',
 'échange',
 'échec',
 'échéancier',
 'éclater',
 'écoute',
 'écran',
 'écrire',
 'également',
 'électricité',
 'électronique',
 'élevé',
 'éléments',
 'épaule',
 'épée',
 'équipement',
 'équipements',
 'équipes',
 'équipés',
 'établissements',
 'étages',
 'étape',
 'état',
 'éteindre',
 'étranger',
 'éventuel',
 'éventuels',
 'éviter',
 'œuf']

In [0]:
len(vect_count.get_feature_names()) # taille du vocabulaire

1821

In [0]:
X_train_vectorized_count = vect_count.transform(X_train)
X_train_vectorized_count

<566x1821 sparse matrix of type '<class 'numpy.int64'>'
	with 4767 stored elements in Compressed Sparse Row format>

In [0]:
X_test_vectorized_count = vect_count.transform(X_test)

In [0]:
vect_count_bigrams = CountVectorizer(min_df=5, ngram_range=(1,2), stop_words=sw).fit(X_train)
X_train_vectorized_count_bigrams = vect_count_bigrams.transform(X_train)
X_test_vectorized_count_bigrams = vect_count_bigrams.transform(X_test)

In [0]:
len(vect_count_bigrams.get_feature_names())

277

### Matrice terme document tf-idf

In [0]:
vect_tfidf = TfidfVectorizer(min_df =5 , stop_words = sw)
vect_tfidf.fit(X_train)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=5, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True,
                stop_words=['au', 'aux', 'avec', 'ce', 'ces', 'dans', 'de',
                            'des', 'du', 'elle', 'en', 'et', 'eux', 'il', 'ils',
                            'je', 'la', 'le', 'les', 'leur', 'lui', 'ma',
                            'mais', 'me', 'même', 'mes', 'moi', 'mon', 'ne',
                            'nos', ...],
                strip_accents=None, sublinear_tf=False,
                token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
                vocabulary=None)

In [0]:
len(vect_count.get_feature_names()), len(vect_tfidf.get_feature_names()) # taille du vocabulaire réduit

(1821, 220)

In [0]:
X_train_vectorized_tfidf = vect_tfidf.transform(X_train)
X_test_vectorized_tfidf = vect_tfidf.transform(X_test)

## Classification du texte

### Classifieur naïf bayésien

In [0]:
model_nb = BernoulliNB().fit(X_train_vectorized_tfidf, y_train)
predictions_test = model_nb.predict(X_test_vectorized_tfidf)

In [0]:
accuracy_score(y_test, predictions_test)

0.86

In [0]:
print(classification_report(y_test, predictions_test))

              precision    recall  f1-score   support

           0       0.82      1.00      0.90        65
           1       1.00      0.60      0.75        35

    accuracy                           0.86       100
   macro avg       0.91      0.80      0.83       100
weighted avg       0.88      0.86      0.85       100



### Régression logistique

- Sur le corpus vectorisé pas nombre d'occurences

In [0]:
model_lr = LogisticRegression(solver='lbfgs', max_iter=500).fit(X_train_vectorized_count, y_train)

In [0]:
predictions_test = model_lr.predict(X_test_vectorized_count)

In [0]:
accuracy_score(y_test, predictions_test)

0.92

In [0]:
print(classification_report(y_test, predictions_test))

              precision    recall  f1-score   support

           0       0.90      0.98      0.94        65
           1       0.97      0.80      0.88        35

    accuracy                           0.92       100
   macro avg       0.93      0.89      0.91       100
weighted avg       0.92      0.92      0.92       100



In [0]:
def print_n_strongly_associated_features(vectoriser, model, n):
    feature_names = np.array(vectoriser.get_feature_names())

    for i in range(1):
        class_name = model.classes_[i]
        print("CLASSE {}".format(class_name))
        idx_coefs_sorted = model.coef_[i].argsort() # ordre croissant
        print("Les dix variables ayant l'association négative la plus forte " + 
              "avec la classe {} :\n{}\n".format(class_name,
                                                 feature_names[idx_coefs_sorted[:n]]))
        idx_coefs_sorted = idx_coefs_sorted[::-1] # ordre décroissant
        print("Les dix variables ayant l'association positive la plus forte " +
              "avec la classe {} :\n{}\n"
              .format(class_name,
                      feature_names[idx_coefs_sorted[:n]]))
        print()

In [0]:
print_n_strongly_associated_features(vect_count, model_lr, 10)

CLASSE 0
Les dix variables ayant l'association négative la plus forte avec la classe 0 :
['ça' 'non' 'veux' 'comme' 'quoi' 'cette' 'allez' 'alors' 'va' 'accord']

Les dix variables ayant l'association positive la plus forte avec la classe 0 :
['voyage' 'comment' 'hôtels' 'séjour' 'paiement' 'quelles' 'dois' 'reçu'
 'réservation' 'personne']




- Sur le corpus vectorisé par TF-IDF

In [0]:
model_lr =  LogisticRegression(solver='lbfgs', max_iter=500).fit(X_train_vectorized_tfidf, y_train)# compléter ici# compléter ici# compléter ici
predictions_test =  model_lr.predict(X_test_vectorized_tfidf)

In [0]:
accuracy_score(y_test, predictions_test)

0.93

In [0]:
print(classification_report(y_test, predictions_test))

              precision    recall  f1-score   support

           0       0.91      0.98      0.95        65
           1       0.97      0.83      0.89        35

    accuracy                           0.93       100
   macro avg       0.94      0.91      0.92       100
weighted avg       0.93      0.93      0.93       100



In [0]:
feature_names = np.array(vect_tfidf.get_feature_names())
idx_tfidf_sorted = X_train_vectorized_tfidf.max(0).toarray()[0].argsort()
print("TF-IDF le moins élevé : {}".format(feature_names[idx_tfidf_sorted[:10]]))
print("TF-IDF le plus élevé : {}".format(feature_names[idx_tfidf_sorted[:-11:-1]]))

TF-IDF le moins élevé : ['forfait' '000' '0825' '10h' 'dim' '20h' '21h30' '825' 'sam' '9h']
TF-IDF le plus élevé : ['pourquoi' 'cette' 'où' 'accord' 'chose' 'personne' 'peut' 'peux' 'plus'
 'possibilité']


- Avec le vectoriseur à unigrammes et bigrammes.

In [0]:
model_lr =  LogisticRegression(solver='lbfgs', max_iter=500).fit(X_train_vectorized_count_bigrams, y_train)# compléter ici# compléter ici# compléter ici
predictions_test =  model_lr.predict(X_test_vectorized_count_bigrams)

In [0]:
accuracy_score(y_test, predictions_test)

0.92

In [0]:
print(classification_report(y_test, predictions_test))

              precision    recall  f1-score   support

           0       0.89      1.00      0.94        65
           1       1.00      0.77      0.87        35

    accuracy                           0.92       100
   macro avg       0.95      0.89      0.91       100
weighted avg       0.93      0.92      0.92       100



In [0]:
print_n_strongly_associated_features(vect_count_bigrams, model_lr, 10)

CLASSE 0
Les dix variables ayant l'association négative la plus forte avec la classe 0 :
['ça' 'veux' 'non' 'sans' 'comme' 'va' 'quoi' 'alors' 'cette' 'allez']

Les dix variables ayant l'association positive la plus forte avec la classe 0 :
['voyage' 'hôtels' 'séjour' 'comment' 'quelles' 'paiement' 'personne'
 'problème' 'cas' 'client']




### SVM

In [0]:
model_svm = SVC(kernel='linear', C=0.1).fit(X_train_vectorized_count_bigrams, y_train)
predictions_test = model_svm.predict(X_test_vectorized_count_bigrams)

In [0]:
accuracy_score(y_test, predictions_test)

0.87

In [0]:
print(classification_report(y_test, predictions_test))

              precision    recall  f1-score   support

           0       0.83      1.00      0.91        65
           1       1.00      0.63      0.77        35

    accuracy                           0.87       100
   macro avg       0.92      0.81      0.84       100
weighted avg       0.89      0.87      0.86       100



In [0]:
def best_model_reg_log(X_train, X_test, y_train, y_valid) :
    X_train = X_train.apply(lemmatise_text)
    X_test = X_test.apply(lemmatise_text)
    sw = nltk.corpus.stopwords.words('french')
    sw += ['être', 'avoir']
    vect_t1 = CountVectorizer(min_df=5, ngram_range=(1,2), stop_words=sw).fit(X_train)
    vect_t1.fit(X_train)
    X_train_vectorized = vect_t1.transform(X_train)
    X_test_vectorized = vect_t1.transform(X_test)
    model_lr = LogisticRegression(solver='lbfgs', max_iter=500).fit(X_train_vectorized, y_train)
    predictions_test =  model_lr.predict(X_test_vectorized)
    return accuracy_score(y_test, predictions_test)
    
best_model_reg_log(X_train, X_test, y_train, y_test)

0.95

In [0]:
def best_model_SVM(X_train, X_test, y_train, y_test) :
    X_train = X_train.apply(lemmatise_text)
    X_test = X_test.apply(lemmatise_text)
    sw = nltk.corpus.stopwords.words('french')
    sw += ['être', 'avoir']
    vect_t1 = CountVectorizer(binary=True,stop_words = sw)
    vect_t1.fit(X_train)
    X_train_vectorized = vect_t1.transform(X_train)
    X_test_vectorized = vect_t1.transform(X_test)
    model_svm3 = SVC(C=10, cache_size=200)
    model_svm3.fit(X_train_vectorized, y_train)
    predictions_test =  model_svm3.predict(X_test_vectorized)
    return accuracy_score(y_test, predictions_test)
    
best_model_SVM(X_train, X_test, y_train, y_test)

0.96

In [0]:
X_train = X_train.apply(lemmatise_text)
X_test = X_test.apply(lemmatise_text)
sw = nltk.corpus.stopwords.words('french')
sw += ['être', 'avoir']
vect_t1 = CountVectorizer(binary=True,stop_words = sw)
vect_t1.fit(X_train)
X_train_vectorized = vect_t1.transform(X_train)
X_test_vectorized = vect_t1.transform(X_test)
model_svm3 = SVC(C=10, cache_size=200)
model_svm3.fit(X_train_vectorized, y_train)
predictions_test =  model_svm3.predict(X_test_vectorized)

## Modèle final

In [0]:
X_total = texts
y_total = labels

### Modèle sur les questions et réponses

Le meilleur modèle obtenu est une SVM. On va l'entrainer sur toutes les données et l'importer.

In [0]:
X_total = pd.DataFrame(X_total)
X_total = X_total[0].apply(lemmatise_text)
sw = nltk.corpus.stopwords.words('french')
sw += ['être', 'avoir']
vect_t1 = CountVectorizer(binary=True,stop_words = sw)
vect_t1.fit(X_total)
X_train_vectorized = vect_t1.transform(X_total)
model_final = SVC(C=10, cache_size=200, probability=True)
model_final.fit(X_train_vectorized, y_total)

SVC(C=10, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=True, random_state=None, shrinking=True, tol=0.001,
    verbose=False)

Exportation du modèle binaire

In [0]:
pickle.dump(model_final, open('classif_binaire_proba', 'wb'))

Exportation du vectorizer

In [0]:
with open("bin_vectorizer", "wb") as fin :
  pickle.dump(vect_t1, fin)

Exportation de la matrice termes documents contenant les questions et les réponses.

In [0]:
with open("bin_matrice_td", "wb") as fin :
  pickle.dump(X_train_vectorized, fin)